# Feature Engineering

In [11]:
# Import libraries
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
import time

In [4]:
# Setup boto and sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
print('Boto and Sagemaker Sessions Initialized...\nBucket: {0}\nRegion: {1}'.format(bucket, region))

Boto and Sagemaker Sessions Initialized...
Bucket: sagemaker-us-east-1-654654380268
Region: us-east-1


In [5]:
database_name = 'foodfacts'
raw_table_name = "aai_540_openfoodfacts"

# Initialize our database name
database_name = 'foodfacts'

# Set S3 staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Create connection to Athena database
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [29]:
# Explore count of food products sold in United States
statement = """SELECT count(*) FROM {}.{} WHERE contains(countries_tags, 'en:united-states');""".format(
    database_name, raw_table_name
)
print(statement)
df_us = pd.read_sql(statement, conn)
df_us

SELECT count(*) FROM foodfacts.aai_540_openfoodfacts WHERE contains(countries_tags, 'en:united-states');


/tmp/ipykernel_363/4075044620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_us = pd.read_sql(statement, conn)


,_col0
0,804173


In [33]:
print(df.columns.tolist())

['additives_n', 'additives_tags', 'allergens_tags', 'brands_tags', 'brands', 'categories', 'categories_tags', 'categories_properties', 'checkers_tags', 'ciqual_food_name_tags', 'cities_tags', 'code', 'compared_to_category', 'complete', 'completeness', 'correctors_tags', 'countries_tags', 'created_t', 'creator', 'data_quality_errors_tags', 'data_quality_info_tags', 'data_quality_warnings_tags', 'data_sources_tags', 'ecoscore_data', 'ecoscore_grade', 'ecoscore_score', 'ecoscore_tags', 'editors', 'emb_codes_tags', 'emb_codes', 'entry_dates_tags', 'food_groups_tags', 'generic_name', 'images', 'informers_tags', 'ingredients_analysis_tags', 'ingredients_from_palm_oil_n', 'ingredients_n', 'ingredients_original_tags', 'ingredients_percent_analysis', 'ingredients_tags', 'ingredients_text', 'ingredients_with_specified_percent_n', 'ingredients_with_unspecified_percent_n', 'ingredients_without_ciqual_codes_n', 'ingredients_without_ciqual_codes', 'ingredients', 'known_ingredients_n', 'labels_tags',

In [50]:
# Select columns

target_col = 'nutriscore_score'

select_cols = [
    'product_name',
    'nutriments',
    'nova_group',
    'additives_n',
    'ingredients_n',
    target_col]



In [51]:
# Filter down dataset to food products in United States with select raw columns
statement = """SELECT 
    product_name,
    nutriments,
    nova_group,
    additives_n,
    ingredients_n,
    nutriscore_score FROM {}.{} WHERE contains(countries_tags, 'en:united-states') LIMIT 5""".format(
    database_name, raw_table_name
)
print(statement)
df_us = pd.read_sql(statement, conn)
df_us

SELECT 
    product_name,
    nutriments,
    nova_group,
    additives_n,
    ingredients_n,
    nutriscore_score FROM foodfacts.aai_540_openfoodfacts WHERE contains(countries_tags, 'en:united-states') LIMIT 5


/tmp/ipykernel_363/885380752.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_us = pd.read_sql(statement, conn)


,product_name,nutriments,nova_group,additives_n,ingredients_n,nutriscore_score
0,"[{'lang': 'main', 'text': 'Simply cottage chee...","[{'name': 'iron', 'value': 0.0, '100g': 0.0, '...",4,1,12,-1
1,"[{'lang': 'main', 'text': 'Dark chocolate bar'...","[{'name': 'fiber', 'value': 9.3, '100g': 9.3, ...",4,1,8,26
2,"[{'lang': 'main', 'text': 'Filled milk chocola...","[{'name': 'trans-fat', 'value': 0.0, '100g': 0...",4,2,20,31
3,"[{'lang': 'main', 'text': 'Filled milk chocola...","[{'name': 'fiber', 'value': 3.5, '100g': 3.5, ...",4,1,21,30
4,"[{'lang': 'main', 'text': 'Dark chocolate truf...","[{'name': 'energy', 'value': 455.0, '100g': 19...",4,2,22,26


In [49]:
df_us.loc[2]['product_name']

[{'lang': 'main', 'text': 'Ice cream'}, {'lang': 'en', 'text': 'Ice cream'}]

In [48]:
df_us.loc[2]['nutriments']

[{'name': 'iron',
  'value': 0.0,
  '100g': 0.0,
  'serving': 0.0,
  'unit': 'mg',
  'prepared_value': None,
  'prepared_100g': None,
  'prepared_serving': None,
  'prepared_unit': None},
 {'name': 'trans-fat',
  'value': 0.0,
  '100g': 0.0,
  'serving': 0.0,
  'unit': 'g',
  'prepared_value': None,
  'prepared_100g': None,
  'prepared_serving': None,
  'prepared_unit': None},
 {'name': 'salt',
  'value': 230.0,
  '100g': 0.23,
  'serving': 0.149,
  'unit': 'mg',
  'prepared_value': None,
  'prepared_100g': None,
  'prepared_serving': None,
  'prepared_unit': None},
 {'name': 'proteins',
  'value': 3.08,
  '100g': 3.08,
  'serving': 2.0,
  'unit': 'g',
  'prepared_value': None,
  'prepared_100g': None,
  'prepared_serving': None,
  'prepared_unit': None},
 {'name': 'cholesterol',
  'value': 23.0,
  '100g': 0.023,
  'serving': 0.0149,
  'unit': 'mg',
  'prepared_value': None,
  'prepared_100g': None,
  'prepared_serving': None,
  'prepared_unit': None},
 {'name': 'energy-kcal',
  'value

In [ ]:
# Parse nutriments column to extract features per 100g

# For example: carbohydrates_100g
# "Negative" Components: energy, saturated fats, sugars, and salt/sodium.
# "Positive" Components: proteins, fiber, and the percentage of fruits, vegetables, and nuts. ...

In [ ]:
# Continue with feature store and feature groups ...